In [2]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import sympy as sym

import models
import payoffs
import plotting
import selection_functions
import symbolics

In [3]:
%matplotlib inline
plt.style.use("ggplot")

# Monomorphic gamma model

Start from full model. Set $x_2=1-x_1, x_3=0$.

In [4]:
x1 = sym.symbols('x1', real=True, nonnegative=True)
T, R, P, S = sym.symbols('T, R, P, S', real=True, positive=True)
M, m = sym.symbols("M, m", real=True, nonnegative=True)

UGA = symbolics.UGA
UgA = symbolics.UgA

# Total offspring (fitness)

In [5]:
x = np.array([[x1], [1 - x1], [0], [0]])
payoff_kernel = np.array([[R, S], [T, P]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M, m)
N, = models.total_offspring(W, x)

In [6]:
N = sym.simplify(N)

In [7]:
N

-2.0*M + 2.0*P*UGA(x1)**2 - 4.0*P*UGA(x1) + 2.0*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2.0*T*UGA(x1)**2 + 2.0*T*UGA(x1)

After a bit of algebra we arrive at the following expression for total offspring.

\begin{align}
   % N(x_1) =& 2.0*P*UGA(x1)**2 - 4.0*P*UGA(x1) + 2.0*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2.0*T*UGA(x1)**2 + 2.0*T*UGA(x1) -2.0*M\\
   % N(x_1) =& 2\bigg(PU_{GA}(x_1)^2 - 2PU_{GA}(x_1) + P + RU_{GA}(x_1)^2 - SU_{GA}(x_1)^2 + SU_{GA}(x_1) - TU_{GA}(x_1)^2 + TU_{GA}(x_1) - M\bigg) \\
   % N(x_1) =& 2\bigg(RU_{GA}(x_1)^2 + PU_{GA}(x_1)^2 - TU_{GA}(x_1)^2 - SU_{GA}(x_1)^2 + SU_{GA}(x_1) + TU_{GA}(x_1) - 2PU_{GA}(x_1) + (P - M)\bigg) \\
    N(x_1) =& 2\bigg(\big((R + P) - (T + S)\big)U_{GA}(x_1)^2 + \big((T + S) - 2P\big)U_{GA}(x_1) + (P - M)\bigg) \\
\end{align}

## Equilibrium total offspring

Consider an equilibrium $x^*$ and define the constant equilibrium selection probability as $U_{GA}^* = U_{GA}(x^*)$.

In [8]:
UGA_star = sym.symbols("UGA_star")
equilibrium_total_offspring = N.subs({UGA(x1): UGA_star})

In [9]:
equilibrium_total_offspring

-2.0*M + 2.0*P*UGA_star**2 - 4.0*P*UGA_star + 2.0*P + 2.0*R*UGA_star**2 - 2.0*S*UGA_star**2 + 2.0*S*UGA_star - 2.0*T*UGA_star**2 + 2.0*T*UGA_star

## Maximizing equilibrium total offspring

Find the value of the equilibrium selection probability $U_{GA}^*$ that maximizes total offspring.

### Find the optimal value of $U_{GA}^*$.

The number of total offspring can be written as a function of the equilibrium selection probability.

$$ N(U_{GA}*) = 2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + (P - M) \bigg)$$

To find the equilibrium selection probability that maximizes the number of total offspring we need to solve the following constrained optimization problem.

$$ \max_{U_{GA}^*}\ 2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + (P - M) \bigg) $$

subject to the following inequality constraints.

\begin{align}
    -U_{GA}^* \le& 0 \\
    U_{GA}^* - 1 \le& 0
\end{align}

First-order condition is as follows.

$$ 4\big((R + P) - (T + S)\big)U_{GA} + 2\big((T + S) - 2P\big) = -\mu_0 + \mu_1 $$

Complementary slackness conditions are

\begin{align}
    -\mu_0U_{GA}^* =& 0 \\
    \mu_1U_{GA}^* - 1 =& 0
\end{align}

where $\mu_0, \mu_1$ are Lagrange multipliers.

#### Case 1: $\mu_0=0, \mu_1>0$; total offspring is maximized by $U_{GA}^*=1$

Substituting $U_{GA}^*=1$ into the first order condition yields

$$ 4\big((R + P) - (T + S)\big) + 2\big((T + S) - 2P\big) = \mu_1 $$

which reduces to the following.

$$ 4R - 2(T + S) = \mu_1 $$

The value of the Lagrange multiplier is equal to the slope of the objective at the constraint. Since the constraint is binding and the multiplier is positive it must be the case that 

$$ 2R > T + S $$

which tells us that the joint payoff to mutual cooperation must exceed the payoff to hierarchy. Note that the number of offspring at $N(1) = 2(R - M)$.

#### Case 1: $\mu_0>0, \mu_1=0$; total offspring is (locally) maximized by $U_{GA}^*=0$

Substituting $U_{GA}^*=0$ into the first order condition yields

$$ 2\big((T + S) - 2P\big) = -\mu_0 $$

which reduces to the following.

$$ -2\big((T + S) - 2P\big) = \mu_0 $$

The value of the Lagrange multiplier is equal to the slope of the objective at the constraint. Since the constraint is binding and the multiplier is positive it must be the case that 

$$ T + S < 2P $$

which tells us that the payoff to hierarchy must be strictly less than the payoff to mutual exploitation. Note that the number of offspring at $N(0) = 2(P - M)$.


#### Case 1: $\mu_0=0, \mu_1=0$; total offspring is maximized by $0 < U_{GA}^* < 1$

Setting the first order condition equal to zero and solving for the optimal value of $U_{GA}^*$ yield the following result.

$$ \bar{U}_{GA}^* = \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

Second order condition for this interior maximum requires that the following payoff restriction holds. 

$$ R + P < T + S $$

The total offspring produced at this optimal equilibrium selection probability is as follows.

$$ N(\bar{U}_{GA}^*) = 2(P-M) + \frac{1}{2}\left(\frac{\big(2P - (T + S)\big)^2}{(T + S) - (R + P)}\right) $$

An interior maximum requires that the slope of the objective at the lower bound be strictly positive and that the slope of the obejctive at the upper bound be strictly negative.  This imposes the additional restrictions on payoffs.

$$ 2P < 2R < T + S $$

Again, we have that the payoff to hierarchy must strictly dominate the payoff to mutual cooperation.

### Discussion

Global maximum number of offspring will either be $N(1)=2R$ or will be an interior maximum with $N(U_{GA}^*) > 2R$. If second order condition for interior maximum fails (i.e., $R + P > T + S$), then interior optimimum is actually a global minimum! In this case while the global maximum is still $U_{GA}^*=1$, there is a local maxmium at $U_{GA}^*=0$.  It might be interesting to explore the invadability of both of these points when $R + P > T + S$.

#### Conditions under which $N(U_{GA}^*)$ is monotonically increasing and convex
$N'(U_{GA}^*) > 0$ requires that 

$$ U_{GA}^* > \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

while $N''(U_{GA}^*) > 0$ requires that

$$ (R + P) > (T + S) > 0. $$

Monotonically increasing and convex $N(U_{GA}^*)$ implies global maximum occurs at $U_{GA}^*=1$ (and no local maximum at $U_{GA}^*=0$).

#### Conditions under which $N(U_{GA}^*)$ is monotonically increasing and concave
$N'(U_{GA}^*) > 0$ requires that 

$$ U_{GA}^* > \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

while $N''(U_{GA}^*) < 0$ requires that

$$ (R + P) < (T + S)$$

Monotonically increasing and concave $N(U_{GA}^*)$ implies global maximum occurs at $U_{GA}^*=1$ (and no local maximum at $U_{GA}^*=0$).


In [10]:
# solve for the interior value of UGA* that maximizes total offspring
foc = equilibrium_total_offspring.diff(UGA_star, 1)
optimal_UGA_star, = sym.solve(foc, UGA_star)

In [11]:
optimal_UGA_star

(P - 0.5*S - 0.5*T)/(P + R - S - T)

In [12]:
_equilibrium_total_offspring = sym.lambdify((UGA_star, T, R, P, S, M),
                                            equilibrium_total_offspring,
                                            modules="numpy")

def plot_total_offspring(T, R, P, S, M):
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    us = np.linspace(0, 1, 100)
    ax.plot(us, _equilibrium_total_offspring(us, T, R, P, S, M))
    ax.set_title(r"Total number of offspring", fontsize=25)
    ax.set_xlabel(r"$U_{GA}^*$", fontsize=25)
    ax.set_ylabel(r"N", rotation="horizontal", fontsize=25)
    plt.show()

In [13]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control metabolic costs
M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")

w = widgets.interactive(plot_total_offspring, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider)
display(w)

A Jupyter Widget

In [14]:
(f,), *_ = models.offspring_genotypes_evolution(W, x)

In [15]:
f

-x1 + (x1*(0.5*(2*(-M + P)*(-UGA(x1) + 1) + 2*(-M + T)*UGA(x1))*(-UGA(x1) + 1) + 1.0*(2*(-M + R)*UGA(x1) + 2*(-M + S)*(-UGA(x1) + 1))*UGA(x1)) + 0.5*(-x1 + 1)*(2*(-M + R)*UGA(x1) + 2*(-M + S)*(-UGA(x1) + 1))*UGA(x1))/(0.5*x1*(2*(-M + P)*(-UGA(x1) + 1) + 2*(-M + T)*UGA(x1))*(-UGA(x1) + 1) + x1*(0.5*(2*(-M + P)*(-UGA(x1) + 1) + 2*(-M + T)*UGA(x1))*(-UGA(x1) + 1) + 1.0*(2*(-M + R)*UGA(x1) + 2*(-M + S)*(-UGA(x1) + 1))*UGA(x1)) + 0.5*(-x1 + 1)*(2*(-M + R)*UGA(x1) + 2*(-M + S)*(-UGA(x1) + 1))*UGA(x1) + (-x1 + 1)*(1.0*(2*(-M + P)*(-UGA(x1) + 1) + 2*(-M + T)*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*(-M + R)*UGA(x1) + 2*(-M + S)*(-UGA(x1) + 1))*UGA(x1)))

# The locus of potential equilibria (LPE)

In [16]:
# solving for the LPE using Python
x1_star, = sym.solve(f, x1, implicit=True)
x1_star =  sym.cancel(x1_star.subs({UGA(x1): UGA_star}))
x1_star

1.0*(2.0*M*UGA_star - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star)/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star)

Setting the equation of motion equal to zero, rearranging the resulting equation yields the locus of potential equilibria (LPE). The LPE represents, for each possible $x_1$, the value that $U_{GA}$ would have to take in order for that $x_1$ to be an equilbrium. 

\begin{align}
  L(x^*) \equiv x_1^* - 2\left(\frac{\big(R - S\big)U_{GA}^{*2} + \big(S - M\big)U_{GA}^*}{N(U_{GA}^*)}\right) = 0\\
\end{align}

We already know that $x_1^*=0, U_{GA}^*=0$ and $x_1^*=1, U_{GA}^*=1$ are equilibria, so it follows that $L(0)=0$ and $L(1)=0$.


Differentiate $x_1^*$ with respect to $U_{GA}^*$ yields an expression that will be useful for assessing stability and invadability of an equilibrium.

In [18]:
dx1star_dUGA_star = sym.factor(sym.cancel(sym.diff(x1_star, UGA_star, 1)))

In [19]:
dx1star_dUGA_star

1.0*(1.0*M**2 + 1.0*M*P*UGA_star**2 - 1.0*M*P + 1.0*M*R*UGA_star**2 - 2.0*M*R*UGA_star - 1.0*M*S*UGA_star**2 + 2.0*M*S*UGA_star - 1.0*M*S - 1.0*M*T*UGA_star**2 - 2.0*P*R*UGA_star**2 + 2.0*P*R*UGA_star + 1.0*P*S*UGA_star**2 - 2.0*P*S*UGA_star + 1.0*P*S + 1.0*R*T*UGA_star**2)/(1.0*M - 1.0*P*UGA_star**2 + 2.0*P*UGA_star - 1.0*P - 1.0*R*UGA_star**2 + 1.0*S*UGA_star**2 - 1.0*S*UGA_star + 1.0*T*UGA_star**2 - 1.0*T*UGA_star)**2

Substituting the equilibrium value for $U_{GA}^*$ that maximizes total offspring, $\bar{U}_{GA}^*$, into the expression for $x_1^*$ we can find the equilibrium value of $x_1^*$ that maximizes total offspring.

\begin{align}
   % \frac{(P - \frac{1}{2}S - \frac{1}{2}T)(PR + \frac{1}{2}RS - \frac{1}{2}RT - \frac{1}{2}S^2 - \frac{1}{2}ST)}{((\frac{1}{2}P + \frac{1}{2}R - \frac{1}{2}S - \frac{1}{2}T)(2PR - \frac{1}{2}S^2 - ST - \frac{1}{2}T^2))} \\
   % \frac{\big(2P - (T + S)\big)(PR + \frac{1}{2}RS - \frac{1}{2}RT - \frac{1}{2}S^2 - \frac{1}{2}ST)}{\big((R + P) - (T + S)\big)(2PR - \frac{1}{2}S^2 - ST - \frac{1}{2}T^2)} \\
   % \frac{\big(2P - (T + S)\big)\big(2PR + RS - RT - S^2 - ST\big)}{\big((R + P) - (T + S)\big)\big(4PR - S^2 - 2ST - T^2\big)} \\
   % \left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right)\left(\frac{2PR + RS - RT - S^2 - ST}{4PR - S^2 - 2ST - T^2}\right) \\
   % \left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right)\left(\frac{2PR + RS - RT - S^2 - ST}{4PR - (T + S)^2}\right) \\
   % \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right)\left(\frac{2(2PR + RS - RT - S^2 - ST)}{4PR - (T + S)^2}\right) \\
   % \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right)\left(\frac{4PR + 2RS - 2RT - 2S^2 - 2ST}{4PR - (T + S)^2}\right) \\
   % \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right)\left(\frac{4PR + 2RS - 2RT -4RS + 4RS - 2S^2 - 2ST}{4PR - (T + S)^2}\right) \\
   % \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right)\left(\frac{2R\big(2P + S - T - 2S\big) + 2S\big(2R - S - T\big)}{4PR - (T + S)^2}\right) \\
   % \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right)\left(\frac{2R\big(2P - (T + S)\big) + 2S\big(2R - (T + S)\big)}{4PR - (T + S)^2}\right) \\
    \bar{x}_1^* = \left(\frac{2R\big(2P - (T + S)\big) + 2S\big(2R - (T + S)\big)}{4PR - (T + S)^2}\right)\bar{U}_{GA}^*
\end{align}

In [20]:
optimal_x1_star = sym.factor(sym.cancel(x1_star.subs({UGA_star: optimal_UGA_star})))

In [21]:
optimal_x1_star

1.0*(1.0*P - 0.5*S - 0.5*T)*(1.0*M*P + 1.0*M*R - 1.0*M*S - 1.0*M*T - 1.0*P*R - 0.5*R*S + 0.5*R*T + 0.5*S**2 + 0.5*S*T)/((0.5*P + 0.5*R - 0.5*S - 0.5*T)*(2.0*M*P + 2.0*M*R - 2.0*M*S - 2.0*M*T - 2.0*P*R + 0.5*S**2 + 1.0*S*T + 0.5*T**2))

## Interactive plot of the locus of potential equilibria

In [22]:
_locus_of_potential_equilibria = sym.lambdify((UGA_star, T, R, P, S, M),
                                              x1_star,
                                              modules="numpy")


def plot_locus_of_potential_equilibria(selection_function, d1, T, R, P, S, M):
    assert T > R > P > S
    us = np.linspace(0, 1, 100)
    xs = _locus_of_potential_equilibria(us, T, R, P, S, M)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    ax.plot(xs, us, label="LPE")
    ax.set_xlabel(r"$x_1^*$", fontsize=25)
    ax.set_ylabel(r"$U_{GA}^*$", rotation="horizontal", fontsize=25)
    ax.plot(us, us, 'k--')
    
    # create the selection functions
    if selection_function == "kirkpatrick":
        UGA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d1)
    elif selection_function == "seger":
        UGA = lambda x_A: selection_functions.seger_selection(x_A, d1)
    elif selection_function == "wright":
        UGA = lambda x_A: selection_functions.wright_selection(x_A, d1)
    else:
        valid_funcs = ("kirkpatrick", "seger", "wright")
        msg = "Selection_function must be one of {}, {}, or {}.".format(*valid_funcs)
        raise ValueError(msg)
        
    # add selection functions for comparison
    ax.plot(xs, UGA(xs), label=selection_function)
    ax.legend()
    
    plt.show()

In [23]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control metabolic costs
M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")

# slider used to control which selection function is being used
UGA_slider = widgets.Dropdown(options=["kirkpatrick", "seger", "wright"], index=0, description=r"$U_{\gamma(j)A}$")

# slider that controls the parameters of the selection function
d1_slider = widgets.FloatSlider(value=1.0, min=0.0, max=10, step=0.05, description=r"$d_1$")

w = widgets.interactive(plot_locus_of_potential_equilibria, selection_function=UGA_slider, d1=d1_slider,
                        T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider)
display(w)

A Jupyter Widget

# Stability

Necessary and sufficient conditions for stability require that the derivative of the equation of motion with respect to $x_1$ be strictly negative when evaluated at the potential equilibrium.

In [24]:
f_jac = f.diff(x1, 1)

In [25]:
sym.powsimp(sym.cancel(f_jac))

(4.0*M**2*U_prime(x1) - 4.0*M**2 + 4.0*M*P*UGA(x1)**2*U_prime(x1) + 8.0*M*P*UGA(x1)**2 - 16.0*M*P*UGA(x1) - 4.0*M*P*U_prime(x1) + 8.0*M*P + 4.0*M*R*UGA(x1)**2*U_prime(x1) + 8.0*M*R*UGA(x1)**2 - 8.0*M*R*UGA(x1)*U_prime(x1) - 4.0*M*S*UGA(x1)**2*U_prime(x1) - 8.0*M*S*UGA(x1)**2 + 8.0*M*S*UGA(x1)*U_prime(x1) + 8.0*M*S*UGA(x1) - 4.0*M*S*U_prime(x1) - 4.0*M*T*UGA(x1)**2*U_prime(x1) - 8.0*M*T*UGA(x1)**2 + 8.0*M*T*UGA(x1) - 4.0*P**2*UGA(x1)**4 + 16.0*P**2*UGA(x1)**3 - 24.0*P**2*UGA(x1)**2 + 16.0*P**2*UGA(x1) - 4.0*P**2 - 8.0*P*R*UGA(x1)**4 + 16.0*P*R*UGA(x1)**3 - 8.0*P*R*UGA(x1)**2*U_prime(x1) - 8.0*P*R*UGA(x1)**2 + 8.0*P*R*UGA(x1)*U_prime(x1) + 8.0*P*S*UGA(x1)**4 - 24.0*P*S*UGA(x1)**3 + 4.0*P*S*UGA(x1)**2*U_prime(x1) + 24.0*P*S*UGA(x1)**2 - 8.0*P*S*UGA(x1)*U_prime(x1) - 8.0*P*S*UGA(x1) + 4.0*P*S*U_prime(x1) + 8.0*P*T*UGA(x1)**4 - 24.0*P*T*UGA(x1)**3 + 24.0*P*T*UGA(x1)**2 - 8.0*P*T*UGA(x1) - 4.0*R**2*UGA(x1)**4 + 8.0*R*S*UGA(x1)**4 - 8.0*R*S*UGA(x1)**3 + 8.0*R*T*UGA(x1)**4 - 8.0*R*T*UGA(x1)**3

### Equilibrium with $x_1^*=0$

In [26]:
sym.simplify(f_jac.subs({x1: 0}))

-0.500000000000000

Assuming that $M < S < P$ then we have the following stability condition for the equilibrium at $x_1^*=0$.

$$ U_{GA}'(0) < \frac{P-M}{S-M} $$

Note that if $M \ge S$, then the equilibrium at $x_1^*=0$ is always stable.

### Equilibrium with $x_1^* = 1$

In [27]:
sym.simplify(f_jac.subs({x1: 1}))

0.5*(R - T)/(M - R)

$$ U_{GA}'(1) < \frac{R - M}{T - M} $$

### Equilibrium with $0 < x^* < 1$

Sufficient conditions to guarantee the existence of a stable, interior equilibrium are that the stability conditions for both corner equilibria are violated. Necessary and sufficient conditions are more difficult to state...

In [28]:
UGA_prime_star = sym.symbols("UGA_prime_star")
evaluated_f_jac = f_jac.subs({sym.Derivative(UGA(x1), x1): UGA_prime_star, UGA(x1): UGA_star, x1: x1_star})

Stability of the equilibrium involves determining whether the following function of $U_{GA}^*$, $U_{GA}^{'*}$ and payoffs $T,R,P,S$ is negative! 

In [29]:
evaluated_f_jac

(0.5*UGA_star*(2*UGA_star*(-M + R) + 2*(-M + S)*(-UGA_star + 1))*(-1.0*(2.0*M*UGA_star - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star)/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star) + 1) + 1.0*(1.0*UGA_star*(2*UGA_star*(-M + R) + 2*(-M + S)*(-UGA_star + 1)) + 0.5*(-UGA_star + 1)*(2*UGA_star*(-M + T) + 2*(-M + P)*(-UGA_star + 1)))*(2.0*M*UGA_star - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star)/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star))*(-0.5*UGA_star*(2*(-M + R)*U_prime(1.0*(2.0*M*UGA_star - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star)/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star)) - 2*(-M + S)*U_prime(1.0*(2.0*M*UGA_star - 2.0*R*UGA_star**2 + 2.0*S*UGA_st

In [30]:
sym.factor(sym.cancel(sym.together(evaluated_f_jac)))

0.5*(1.0*M**2*U_prime(2.0*M*UGA_star/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star) - 2.0*R*UGA_star**2/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star) + 2.0*S*UGA_star**2/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star) - 2.0*S*UGA_star/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star)) - 1.0*M**2 + 1.0*M*P*UGA_star**2*U_prime(2.0*M*UGA_star/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 2.0*S*UGA_star + 2.0*T*UGA_star**2 - 2.0*T*UGA_star) - 2.0*R*UGA_star**2/(2.0*M - 2.0*P*UGA_star**2 + 4.0*P*UGA_star - 2.0*P - 2.0*R*UGA_star**2 + 2.0*S*UGA_star**2 - 

In [31]:
# the above expression can be reduce to a ratio of two, two-dimensional polynomial functions in U and U'...
numerator, denominator = sym.fraction(sym.factor(sym.cancel(sym.together(evaluated_f_jac))))

Can we prove that the denominator is strictly positive? If so, then we have reduced the problem of finding the sign of the Jacobian to finding the sign of the expression in the numerator. Yes!

In [32]:
sym.factor(denominator)

1.0*(1.0*M - 1.0*P*UGA_star**2 + 2.0*P*UGA_star - 1.0*P - 1.0*R*UGA_star**2 + 1.0*S*UGA_star**2 - 1.0*S*UGA_star + 1.0*T*UGA_star**2 - 1.0*T*UGA_star)**2

Now we can solve the numerator for $U'_{GA}(x^*)$.  This will allow us to write the condition for which the Jacobian will be negative as an upper bound on $U'_{GA}(x^*)$.

In [34]:
upper_bound, = sym.solve(numerator, UGA_prime_star)

ValueError: not enough values to unpack (expected 1, got 0)

In [ ]:
sym.factor(upper_bound)

After fiddling around with the above expression we arrive at the following condition for a negative Jacobian and a stable interior equilibrium at $0 < x^* < 1$.

\begin{align}
   % U'_{GA}(x^*) <& \frac{(M - PU_{GA}^{*2} + 2PU_{GA}^* - P - RU_{GA}^{*2} + SU_{GA}^{*2} - SU_{GA}^* + TU_{GA}^{*2} - TU_{GA}^*)^2}{(M^2 + MPU_{GA}^{*2} - MP + MRU_{GA}^{*2} - 2MRU_{GA}^* - MSU_{GA}^{*2} + 2MSU_{GA}^* - MS - MTU_{GA}^{*2} - 2PRU_{GA}^{*2} + 2PRU_{GA}^* + PSU_{GA}^{*2} - 2PSU_{GA}^* + PS + RTU_{GA}^{*2})} \\
   % U'_{GA}(x^*) <& \frac{\bigg(-1(-M + PU_{GA}^{*2} - 2PU_{GA}^* + P + RU_{GA}^{*2} - SU_{GA}^{*2} + SU_{GA}^* - TU_{GA}^{*2} + TU_{GA}^*)\bigg)^2}{(M^2 + MPU_{GA}^{*2} - MP + MRU_{GA}^{*2} - 2MRU_{GA}^* - MSU_{GA}^{*2} + 2MSU_{GA}^* - MS - MTU_{GA}^{*2} - 2PRU_{GA}^{*2} + 2PRU_{GA}^* + PSU_{GA}^{*2} - 2PSU_{GA}^* + PS + RTU_{GA}^{*2})} \\
   % U'_{GA}(x^*) <& \frac{\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + \big(P - M\big)\bigg)^2}{MRU_{GA}^{*2} + MPU_{GA}^{*2} - MSU_{GA}^{*2} - MTU_{GA}^{*2} - 2PRU_{GA}^{*2} + PSU_{GA}^{*2} + RTU_{GA}^{*2} - 2MRU_{GA}^* + 2MSU_{GA}^* + 2PRU_{GA}^* - 2PSU_{GA}^* + PS - MS - MP + M^2 }\\
   % U'_{GA}(x^*) <& \frac{\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + \big(P - M\big)\bigg)^2}{\bigg(M\big((R + P) - (T + S)\big) + \big(TR + PS - 2RP\big)\bigg)U_{GA}^{*2} + 2(R - S)(P - M)U_{GA}^* + \big(PS - M(P + S) + M^2\big) }\\
    U'_{GA}(x^*) <& \frac{\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + \big(P - M\big)\bigg)^2}{\bigg(M\big((R + P) - (T + S)\big) + \big(TR + PS - 2RP\big)\bigg)U_{GA}^{*2} + 2(R - S)(P - M)U_{GA}^* + (P - M)(S - M)}\\
\end{align}

We should be able to recover the stability conditions for the corner equilibria as a special case of the above condition. Recall that $U_{GA}(0) = 0$ and $U_{GA}(1)=1$. When $x^*=0$ the condition reduces to $U'_{GA}(0) < \frac{P - M}{S - M}$ and when $x^*=1$ the condition reduces to $U'_{GA}(1) < \frac{R - M}{T - M}$.

Can also use a graphical approach to check the correctness of the above condition.  The above condition should be the zero contour in a contour plot of the value of the Jacobian as a function of $U_{GA}$ and $U'_{GA}$.

In [ ]:
# vectorized numerical function created from our symbolic expression
_numeric_f_jac = sym.lambdify((UGA_star, UGA_prime_star, T, R, P, S, M), evaluated_f_jac, modules="numpy")

In [ ]:
def plot_jacobian(T, R, P, S, M):

    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{GA}^{'*}$", fontsize=20)
    ax.set_title(r"Negative values of $\frac{\partial \dot{x}_1}{\partial x_1}$ indicate stability!", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    equilibrium_selection_derivs = np.linspace(0, 10, 1000).reshape(1, -1) # sensible UGA'(0) >= 1
    Z = _numeric_f_jac(equilibrium_selection_probs, equilibrium_selection_derivs, T, R, P, S, M)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=-0.5, vmax=1.0)

    levels = np.arange(-0.25, 1.25, 0.25)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 10, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [ ]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")

w = widgets.interactive(plot_jacobian, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider)
display(w)

# Invadability

Restrict the full model to obtain a monomorphic $\gamma$ equilibrium, add a small amount of individuals carrying the $g$ allele of the $\gamma$ gene and simulate.

In [ ]:
x2, x3 = sym.symbols("x2, x3", real=True, nonnegative=True)
x4 = 1 - x1 - x2 - x3
x = np.array([[x1], [x2], [x3], [x4]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M, m)
f1, f2, f3, _ = models.offspring_genotypes_evolution(W, x)

In [ ]:
F = sym.Matrix([f1, f2, f3])

In [ ]:
F_jac = F.jacobian([x1, x2, x3])

In [ ]:
UgA_star = sym.symbols("UgA_star", nonnegative=True, real=True)

evaluated_F_jac = (F_jac.subs({x2: 1 - x1, x3: 0})
                        .subs({UGA(x1): UGA_star, UgA(x1): UgA_star})
                        .doit()
                        .subs({sym.Derivative(UGA(x1), x1): UGA_prime_star})
                        .subs({x1: x1_star}))

In [ ]:
simplified_F_jac = sym.zeros(3, 3)

for i in range(3):
    for j in range(3):
        simplified_F_jac[i, j] = sym.factor(sym.cancel(sym.together(evaluated_F_jac[i, j])))
        print("Finished with element {},{}!".format(i,j))
        

In [ ]:
eigenvals = simplified_F_jac.eigenvals()

In [ ]:
e1, e2, e3 = eigenvals.keys()

In [ ]:
e1

In [ ]:
sym.factor(e1)

First eigenvalue can be written as a hyperbola in $U_{gA}^*,U_{GA}^*$ plane.

$$ e_1 = \left(\frac{\big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^*}{N^*}\right) - \left(\frac{\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^*}{N^*}\right) $$

### Completing the square

To write this hyperbola in standard form we need to [complete the square](https://en.wikipedia.org/wiki/Completing_the_square). Completing the square for the quadratic polynomial in $U_{GA}(x^*)$ yields the following.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{GA}^* - \bar{U}_{GA}^*\right)^2 - \frac{1}{2}\big(\bar{N}^* - 2P\big) \\
\end{align}

where $\bar{U}_{GA}^*$ is the value of $U_{GA}^*$ that maximizes total offspring $N$ derived above. Completing the square for the quadratic polynomial in $U_{gA}(x^*)$ yields a similar expression.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{gA}^* - \bar{U}_{GA}^*\right)^2 - \frac{1}{2}\big(\bar{N}^* - 2P\big) \\
\end{align}

Substituting these results into the expression for the eigenvalue yields the following.

$$ e_1 = \left(\frac{\big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) - \left(\frac{\big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) $$


### Conditions for negative eigenvalue

Non-invadability requires that this eigenvalue is strictly negative.

$$ \left(\frac{\big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) - \left(\frac{\big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) < 0$$

#### Case: (R + P) < (T + S)
In this case the inequality simplifies to the following.

$$ \big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2} - \big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2} > 0 $$

This implies that we have four sub-cases to consider.

\begin{align}
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) >& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) >& -\big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) >& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) >& -\big(U_{GA}^* - \bar{U}_{GA}^*\big)
\end{align}

These four sub-cases define the following two regions of the $U_{gA}^*,U_{GA}^*$ plane where the eigenvalue is negative.

\begin{align}
   & 2\big(\bar{U}_{GA}^* - U_{gA}^*\big) < U_{GA}^* - U_{gA}^* < 0\\
   & 0 < U_{GA}^* - U_{gA}^* < 2\big(\bar{U}_{GA}^* - U_{gA}\big)
\end{align}

#### Case: (R + P) > (T + S)
In this case the inequality simplifies to the following.

$$ \big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2} - \big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2} < 0 $$

This implies that we again have four sub-cases to consider.

\begin{align}
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) <& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) <& -\big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) <& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) <& -\big(U_{GA}^* - \bar{U}_{GA}^*\big)
\end{align}

These four sub-cases define the following two regions of the $U_{gA}^*,U_{GA}^*$ plane where the eigenvalue is negative.

\begin{align}
   & 2\big(\bar{U}_{GA}^* - U_{gA}^*\big) < U_{GA}^* - U_{gA}^* < 0\\
   & 0 < U_{GA}^* - U_{gA}^* < 2\big(\bar{U}_{GA}^* - U_{gA}\big)
\end{align}


### Asymptotes 

Asymptotes of this hyperbola can be defined as follows.

$$ U_{GA}^* - \bar{U}_{GA}^* = \pm\big(U_{gA}^* - \bar{U}_{GA}^*\big) $$

The above can be written as linear equations in $U_{gA}^*,U_{GA}^*$ plane.

\begin{align}
    U_{GA}^* =& U_{gA}^* \\
    U_{GA}^* =& 2\bar{U}_{GA}^* - U_{gA}^*
\end{align}

Note that the asymptotes of this hyperbola correspond to the locus of points for which the value of the first eigenvalue is exactly zero.

#### Discussion

Suppose $U_{GA}^* = \bar{U}_{GA}^*$, then both of the sets of inequalities will hold for any value of $U_{gA}^*$. This implies that the eigenvalue will be negative for any value of $U_{gA}^*$.

In [ ]:
# vectorized numeric repr for the eigenvalue
_numeric_e1 = sym.lambdify((UGA_star, UGA_prime_star, T, R, P, S, M), e1, modules="numpy")

In [ ]:
def plot_first_eigenvalue(T, R, P, S, M):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{GA}^{'*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_1$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    equilibrium_selection_derivs = np.linspace(0, 10, 1000).reshape(1, -1) # sensible UGA'(0) >= 1
    Z = _numeric_e1(equilibrium_selection_probs, equilibrium_selection_derivs, T, R, P, S, M)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=-0.5, vmax=1.0)

    levels = np.arange(-0.25, 1.25, 0.25)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 10, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [ ]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")

w = widgets.interactive(plot_first_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider)
display(w)

In [ ]:
UGA_prime_star, = sym.solve(numerator, UGA_prime_star)

In [ ]:
sym.factor(UGA_prime_star)

After a bit of fiddling, we arrive at the same stability condition derived above.

\begin{align}
   % U'_{GA}(x^*) < -\frac{(PU_{GA}(x^*)^2 - 2PU_{GA}(x^*) + P + RU_{GA}(x^*)^2 - SU_{GA}(x^*)^2 + SU_{GA}(x^*) - TU_{GA}(x^*)^2 + TU_{GA}(x^*))^2}{(2PRU_{GA}(x^*)^2 - 2PRU_{GA}(x^*) - PSU_{GA}(x^*)^2 + 2PSU_{GA}(x^*) - PS - RTU_{GA}(x^*)^2)} \\
    U'_{GA}(x^*) < \frac{\bigg(\big((R + P) - (T + S)\big)U_{GA}(x^*)^2 + \big((T + S) - 2P\big)U_{GA}(x^*) + P\bigg)^2}{\big(R(T-2P) + PS\big)U_{GA}(x^*)^2 + 2P(R - S)U_{GA}(x^*) + PS}
\end{align}

In [ ]:
e2

$$ e_2 = \left(\frac{\big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^* + P}{4N^*}\right) - \frac{3}{4} $$

### Completing the square

Completing the square for the quadratic polynomial in $U_{gA}(x^*)$ yields the following expression.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{gA}^* - \bar{U}_{GA}^*\right)^2 + \frac{1}{2}\bar{N}^* \\
\end{align}

Finally we need to find conditions under which the third eigenvalue is strictly negative.

In [ ]:
# vectorized numeric repr for the eigenvalue
_numeric_e2 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m), e2, modules="numpy")

In [ ]:
def plot_second_eigenvalue(T, R, P, S, M, m):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_2$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e2(UGAs, UgAs, T, R, P, S, M, m)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [ ]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m$")

w = widgets.interactive(plot_second_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider, m=m_slider)
display(w)

In [ ]:
e3

In [ ]:
# vectorized numeric repr for the eigenvalue
_numeric_e3 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m), e3, modules="numpy")

In [ ]:
def plot_third_eigenvalue(T, R, P, S, M, m):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_3$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e3(UGAs, UgAs, T, R, P, S, M, m)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [ ]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m$")

w = widgets.interactive(plot_third_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider, m=m_slider)
display(w)

## When is the fitness-maximizing interior equilibrium un-invadable by randomista?

In [ ]:
evaluated_e3 = e3.subs({UGA_star: optimal_UGA_star, UgA_star: optimal_x1_star, m: 0})
simplified_e3 = sym.factor(sym.cancel(sym.together(evaluated_e3)))

In [ ]:
numerator, denominator = sym.fraction(simplified_e3)

In [ ]:
numerator

\begin{align}
    4\bigg(-\big(P^3 + 3P^2R + 3PR^2 + R^3\big) - \big(P^3 - 3P^2S + 3PS^2 + S^3\big) + \big(-P^3 + 3P^2T - 3PT^2 + T^3\big) - \big(R^3 - 3R^2S + 3RS^2 - S^3\big) + \big(T^3 - 3RT^2 + 3R^2T - R^3\big) + 2P^3 + 2R^3 + 8PRS + 8PRT - 8PST - 8RST - 2T^3 + \big(S^3 + 3S^2T + 3ST^2 + T^3\big)\bigg)M^3 + 2\big(4P^3R + 8P^2R^2 - 8P^2RS - 8P^2RT - P^2S^2 - 2P^2ST - P^2T^2 + 4PR^3 - 8PR^2S - 8PR^2T + 2PRS^2 + 4PRST + 2PRT^2 + 2PS^3 + 6PS^2T + 6PST^2 + 2PT^3 - R^2S^2 - 2R^2ST - R^2T^2 + 2RS^3 + 6RS^2T + 6RST^2 + 2RT^3 - S^4 - 4S^3T - 6S^2T^2 - 4ST^3 - T^4\big)M^2 + \big(- 4P^3R^2 - 4P^2R^3 + 4P^2R^2S + 4P^2R^2T + 2P^2RS^2 + 4P^2RST + 2P^2RT^2 + 2PR^2S^2 + 4PR^2ST + 2PR^2T^2 - 2PRS^3 - 6PRS^2T - 6PRST^2 - 2PRT^3 - 0.25PS^4 -PS^3T - 1.5PS^2T^2 - PST^3 - 0.25PT^4 - 0.25RS^4 - RS^3T - 1.5RS^2T^2 - RST^3 - 0.25RT^4 + 0.25S^5 + 1.25S^4T + 2.5S^3T^2 + 2.5S^2T^3 + 1.25ST^4 + 0.25T^5\big)M + \big(- P^2R^2S^2 + 2P^2R^2ST - P^2R^2T^2 + P^2RS^3 - P^2RS^2T - P^2RST^2 + P^2RT^3 - 0.25P^2S^4 + 0.5P^2S^2T^2 - 0.25P^2T^4 + PR^2S^3 - PR^2S^2T - PR^2ST^2 + PR^2T^3 - PRS^4 + 2PRS^2T^2 - PRT^4 + 0.25PS^5 + 0.25PS^4T - 0.5PS^3T^2 - 0.5PS^2T^3 + 0.25PST^4 + 0.25PT^5 - 0.25R^2S^4 + 0.5R^2S^2T^2 - 0.25R^2T^4 + 0.25RS^5 + 0.25RS^4T - 0.5RS^3T^2 - 0.5RS^2T^3 + 0.25RST^4 + 0.25RT^5 - 0.0625S^6 - 0.125S^5T + 0.0625S^4T^2 + 0.25S^3T^3 + 0.0625S^2T^4 - 0.125ST^5 - 0.0625T^6\big) \\
    4\bigg(-\big(R + P\big)^3 - \big(P - S\big)^3 + \big(T - P\big)^3 - \big(R - S\big)^3 + \big(T - R\big)^3 + 2P^3 + 2R^3 + 8PRS + 8PRT - 8PST - 8RST - 2T^3 + \big(T + S\big)^3\bigg)M^3 + 2\big(4P^3R + 8P^2R^2 - 8P^2RS - 8P^2RT - P^2S^2 - 2P^2ST - P^2T^2 + 4PR^3 - 8PR^2S - 8PR^2T + 2PRS^2 + 4PRST + 2PRT^2 + 2PS^3 + 6PS^2T + 6PST^2 + 2PT^3 - R^2S^2 - 2R^2ST - R^2T^2 + 2RS^3 + 6RS^2T + 6RST^2 + 2RT^3 - S^4 - 4S^3T - 6S^2T^2 - 4ST^3 - T^4\big)M^2 + \big(- 4P^3R^2 - 4P^2R^3 + 4P^2R^2S + 4P^2R^2T + 2P^2RS^2 + 4P^2RST + 2P^2RT^2 + 2PR^2S^2 + 4PR^2ST + 2PR^2T^2 - 2PRS^3 - 6PRS^2T - 6PRST^2 - 2PRT^3 - 0.25PS^4 -PS^3T - 1.5PS^2T^2 - PST^3 - 0.25PT^4 - 0.25RS^4 - RS^3T - 1.5RS^2T^2 - RST^3 - 0.25RT^4 + 0.25S^5 + 1.25S^4T + 2.5S^3T^2 + 2.5S^2T^3 + 1.25ST^4 + 0.25T^5\big)M + \big(- P^2R^2S^2 + 2P^2R^2ST - P^2R^2T^2 + P^2RS^3 - P^2RS^2T - P^2RST^2 + P^2RT^3 - 0.25P^2S^4 + 0.5P^2S^2T^2 - 0.25P^2T^4 + PR^2S^3 - PR^2S^2T - PR^2ST^2 + PR^2T^3 - PRS^4 + 2PRS^2T^2 - PRT^4 + 0.25PS^5 + 0.25PS^4T - 0.5PS^3T^2 - 0.5PS^2T^3 + 0.25PST^4 + 0.25PT^5 - 0.25R^2S^4 + 0.5R^2S^2T^2 - 0.25R^2T^4 + 0.25RS^5 + 0.25RS^4T - 0.5RS^3T^2 - 0.5RS^2T^3 + 0.25RST^4 + 0.25RT^5 - 0.0625S^6 - 0.125S^5T + 0.0625S^4T^2 + 0.25S^3T^3 + 0.0625S^2T^4 - 0.125ST^5 - 0.0625T^6\big) \\
\end{align}

In [ ]:
(S - R)(S^2 - 2SR + R^2) = S^3 - 3RS^2 + 3SR^2 - R^3

In [ ]:
sym.factor(-P**3 - 3*P**2*R + 3*P**2*S + 3*P**2*T - 3*P*R**2 + 8*P*R*S + 8*P*R*T - 3*P*S**2 - 8*P*S*T - 3*P*T**2 - R**3 + 3*R**2*S + 3*R**2*T - 3*R*S**2 - 8*R*S*T - 3*R*T**2 + S**3 + 3*S**2*T + 3*S*T**2 + T**3)

In [ ]:
r1, r2, r3 = sym.solve(numerator, M)

In [ ]:
sym.cancel(sym.together(r1))

In [ ]:
denominator

Denominator will be negative if and only if...

\begin{align}
    % 4MP + 4MR - 4MS - 4MT - 4PR + S^2 + 2ST + T^2 < 0 \\
    % 4M\big((R + P) - (T + S)\big) - 4PR + (T + S)^2< 0 \\
    % 4M\big((R + P) - (T + S)\big) < 4RP - (T + S)^2 \\
    M < \frac{1}{4}\frac{4RP - (T + S)^2}{(R + P) - (T + S)} \\
    M < \frac{RP - \left(\frac{T + S}{2}\right)^2}{(R + P) - (T + S)} \\
\end{align}

...since we are looking at interior equilibrium the RHS of the above inequality will be positive.

In [ ]:
_f = sym.lambdify((T, R, P, S, M), evaluated_e3, modules = "numpy")

def plot_non_invadable_by_randomista(T, R, P, S):
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    Ms = np.linspace(0, min(0.5 * (T + S), P), 100)
    ax.plot(Ms, _f(T, R, P, S, Ms))
    ax.set_xlabel(r"$M$", fontsize=15)
    ax.set_ylabel(r"$e_3$", rotation="horizontal", fontsize=15)
    plt.show()

In [ ]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

w = widgets.interactive(plot_non_invadable_by_randomista, T=T_slider, R=R_slider, P=P_slider, S=S_slider)
display(w)

In [ ]:
solutions = sym.solve(simplified_e3, M)